In [ ]:
# Import the speech_recognition to convert the spoken words into text, make a query or give a reply
import speech_recognition as sr
# Import the transformers library to use pre-trained models
from transformers import pipeline
# Import the text-to-speech library
import pyttsx3
# Import the Natural Language Toolkit for text processing
import nltk
# Import function to tokenize text into words
from nltk.tokenize import word_tokenize
# Import list of common stopwords to be removed
from nltk.corpus import stopwords
# Import the stemmer for reducing words to root form
from nltk.stem import PorterStemmer
import nltk
from nltk.data import find
from nltk.downloader import Downloader

# Function to check if a resource is already downloaded
def is_resource_downloaded(resource):
    try:
        find(resource)
        return True
    except LookupError:
        return False

# Download necessary NLTK data files if not already downloaded
if not is_resource_downloaded('tokenizers/punkt'):
    nltk.download('punkt')
if not is_resource_downloaded('corpora/stopwords'):
    nltk.download('stopwords')

# Function to convert speech to text using Google Speech Recognition
def speech_to_text(audio_file):
    r = sr.Recognizer()
    with sr.AudioFile(audio_file) as source:
        audio = r.record(source)
    try:
        # Use Google Speech Recognition to convert audio to text
        text = r.recognize_google(audio)
        print("Speech recognized:", text)
        return text
    # Handle unrecognized speech
    except sr.UnknownValueError:
        print("Speech Recognition could not understand audio")
    # Handle request errors
    except sr.RequestError as e:
        print("Could not request results from Speech Recognition service; {0}".format(e))

# Function to preprocess text by tokenizing, stemming, and removing stopwords
def preprocess_text(text):
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    ps = PorterStemmer()
    # Remove stopwords
    filtered_tokens = [ps.stem(word) for word in tokens if word.lower() not in stop_words]
    # Join the filtered tokens back into a string
    return " ".join(filtered_tokens)

# Function to remove repeated words from text
def remove_repeated_words(text):
    # Split the text into words
    words = text.split()
    seen = set()
    new_words = []
    for word in words:
        if word.lower() not in seen:
            new_words.append(word)
            seen.add(word.lower())
    # Join the new words back into a string
    return " ".join(new_words)

# Function to summarize text using a pre-trained model and remove repeated words
def summarize_text(text):
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
        # Generate the summary
    summary = summarizer(text, max_length=50, min_length=5)
    if 'summary_text' in summary[0]:
        clean_summary = remove_repeated_words(summary[0]['summary_text'])
        return clean_summary
    else:
        # Handle summarization failures
        print("Summarization failed.")
        return "Summarization failed."

# Function to convert text to speech
def text_to_speech(text):
    engine = pyttsx3.init()
    engine.say(text)
    engine.runAndWait()

# Function to handle pre-recorded audio
def process_pre_recorded():
    # Path to the pre-recorded audio file
    audio_file = "C:/Users/saksh/minor_project/speech1.wav"
    print("Processing pre-recorded speech...")
    return audio_file

# Function to record speech from the microphone
def record_speech():
    # Path to save the recorded audio
    audio_file = "C:/Users/saksh/speech.wav"
    print("Recording speech...")
    with sr.Microphone() as source:
        r = sr.Recognizer()
        # Listen to the microphone input
        audio = r.listen(source, timeout=5)
        with open(audio_file, "wb") as f:
            f.write(audio.get_wav_data())
    print("Speech recorded.")
    return audio_file

# Main function to handle user input and process speech
def main():
    while True:
        choice = input('''Enter 1 for pre-recorded audio 
Enter 2 to record the audio now:''')
        # Get user's choice
        if choice == '1':
            # Process pre-recorded audio
            audio_file = process_pre_recorded()
        elif choice == '2':
            # Record new speech
            audio_file = record_speech()
        else:
            print("Invalid choice...Continuing with the pre-recorded audio")
            audio_file = process_pre_recorded()
        # Convert speech to text
        text = speech_to_text(audio_file)
        # If text is recognized
        if text:
            # Preprocess the text
            preprocessed_text = preprocess_text(text)
            # Summarize the text
            summary = summarize_text(preprocessed_text)
              # Ask user preference for summary output
            output_choice = input("Do you want to see the summary as text or hear it spoken? (see/hear/both): ")
            if output_choice.lower() == "see":
                print("Summarized text:", summary)
            elif output_choice.lower() == "hear":
                    # Convert summarized text to speech
                text_to_speech(summary)
            elif output_choice.lower() == "both":
                print("Summarized text:", summary)
                text_to_speech(summary)
            else:
                print("Invalid choice. Summary will be shown as text.")
                print("Summarized text:", summary)
        else:
            # Handle unrecognized speech
            print("Speech could not be recognized. Please try again.")
        # Ask the user if they want to try again
        try_again = input('''Do you want to try again? 
Enter "yes" or "no": ''')
        if try_again.lower() != 'yes':
            print("Exiting the program.")
            break
# Run the main function
if __name__ == "__main__":
    main()
    